## Remote ID Attack

In [16]:
import os
import subprocess
import numpy as np

# Custom modules
from vehicle_logic import VehicleLogic,VehicleMode
from helpers.change_coordinates import heading_to_yaw,find_spawns,global2local,local2global
from helpers.visualization.gazebo_world import update_world
from helpers.visualization.plots import plot_3d_interactive
from plan.actions.navegation import make_go_to
from plan import Plan,State
from gcs import GCS

Kill all related process

In [17]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/runway.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

## Choose Simulator

In [4]:
simulator ='gazebo' # 'QGroundControl' #

## Create Plans

In [ ]:
offsets = [(-10, 0, 0, 0),(0, -5, 0, 0)]#
n_uavs = len(offsets)


homes=np.array([offset[:3] for offset in offsets])

gcs=GCS(positions=homes.copy())
global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
              np.array([[0, -5, 5]])]

local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]


radar_radius=10
avoidance_radius =5


## Remote ID attack
fake_posistion=np.array([[0,0,5]])
markers = {f'waypoints': {'pos':global_paths[0],'color':'blue'},
       f'real pos': {'pos':global_paths[1],'color':'red','radius':0.1},
       f'fake_pos': {'pos':fake_posistion,'color':'orange','radius':0.1},
       f'avoid zone': {'pos':fake_posistion,'color':'red','radius':avoidance_radius,'alpha': 0.85},
       f'radar zone': {'pos':global_paths[1],'color':'orange','radius':radar_radius,'alpha': 0.9}}


# # Normal behavior
# markers = {f'waypoints': {'pos':global_paths[0],'color':'green'},
#        f'real pos': {'pos':global_paths[1],'color':'red','radius':0.1},
#        f'avoid zone': {'pos':global_paths[1],'color':'orange','radius':avoidance_radius,'alpha': 0.84},
#        f'radar zone': {'pos':global_paths[1],'color':'orange','radius':radar_radius,'alpha': 0.9}}





plans=[Plan.basic(wps=local_paths[0],alt=5,wp_margin=1,navegation_speed=1.5),
       Plan.hover(wps=local_paths[1],alt=5,wp_margin=1,navegation_speed=5,verbose=1)]
print(plans)


markers_plot = {f'waypoints': {'pos':global_paths[0],'color':'green'},
                f'uav start': {'pos':np.array([[-10, 0, 0]]),'color':'green'},
                'uav goal': {'pos':np.array([[10, 0, 0]]),'color':'green'},
              f'adversary pos': {'pos':global_paths[1],'color':'red','radius':0.1},
              f'adversary fake pos': {'pos':fake_posistion,'color':'orange','radius':0.1}
              }


plot_3d_interactive(markers_plot,title='Simulation Markers',expand=[0.4,0.4,0.2],ground=-0.05)
fake_posistion=fake_posistion[0]

if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,0)
    spawns=find_spawns(home_position, offsets)

Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Set Navigation Speed' created — no connection yet 🧩
Step 'Set speed to 1.50 m/s' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (20, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'hover' created — no connection yet 🧩
Action 'Pre-Arm 

## Waypoint seletion algorithm

In [6]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

Launch Vehicle

In [7]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)

In [8]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

Create UavLogic

In [9]:
uavs=[]
for i,(plan,home) in enumerate(zip(plans,homes)):
    uavs.append(VehicleLogic(sys_id=i+1,
                    home=home,
                    plan= plan))

vehicle 1 created
vehicle 2 created


In [10]:
plans[0]

🕓 <Plan 'basic'>
  🕓 <Action 'Pre-Arm Check'>
    🕓 <Step 'Check disarmed'>
    🕓 <Step 'Check EKF'>
    🕓 <Step 'Check GPS'>
    🕓 <Step 'Check system'>
  🕓 <Action 'Set Mode: GUIDED'>
    🕓 <Step 'Switch to GUIDED'>
  🕓 <Action 'Set Navigation Speed'>
    🕓 <Step 'Set speed to 1.50 m/s'>
  🕓 <Action 'Arm'>
    🕓 <Step 'arm'>
  🕓 <Action 'takeoff'>
    🕓 <Step 'takeoff'>
  🕓 <Action 'fly'>
    🕓 <Step 'go to  -> (0, 0, 5)'>
    🕓 <Step 'go to  -> (20, 0, 5)'>
  🕓 <Action 'Land'>
    🕓 <Step 'land'>

In [11]:
plans[1]

🕓 <Plan 'hover'>
  🕓 <Action 'Pre-Arm Check'>
    🕓 <Step 'Check disarmed'>
    🕓 <Step 'Check EKF'>
    🕓 <Step 'Check GPS'>
    🕓 <Step 'Check system'>
  🕓 <Action 'Set Mode: GUIDED'>
    🕓 <Step 'Switch to GUIDED'>
  🕓 <Action 'Arm'>
    🕓 <Step 'arm'>
  🕓 <Action 'takeoff'>
    🕓 <Step 'takeoff'>
  🕓 <Action 'fly'>
    🕓 <Step 'go to  -> (0, 0, 5)'>

In [12]:
print(plans[0])

🕓 <Plan 'basic'>
  🕓 <Action 'Pre-Arm Check'>
    🕓 <Step 'Check disarmed'>
    🕓 <Step 'Check EKF'>
    🕓 <Step 'Check GPS'>
    🕓 <Step 'Check system'>
  🕓 <Action 'Set Mode: GUIDED'>
    🕓 <Step 'Switch to GUIDED'>
  🕓 <Action 'Set Navigation Speed'>
    🕓 <Step 'Set speed to 1.50 m/s'>
  🕓 <Action 'Arm'>
    🕓 <Step 'arm'>
  🕓 <Action 'takeoff'>
    🕓 <Step 'takeoff'>
  🕓 <Action 'fly'>
    🕓 <Step 'go to  -> (0, 0, 5)'>
    🕓 <Step 'go to  -> (20, 0, 5)'>
  🕓 <Action 'Land'>
    🕓 <Step 'land'>


## Execution

In [13]:
while gcs.incomplete_missions:
    ## Get global positions
    to_remove = set()
    for id in gcs.incomplete_missions:
        uav=uavs[id-1]
        if uav.current_action().name =='fly':
            gcs.ask_position(uav) 
            gcs.poss[1] = fake_posistion  
    for id in gcs.incomplete_missions:
        uav = uavs[id-1]
        if uav.current_action().name =='fly':
            jd,min_dist=gcs.get_nearest_neighbor(uav)
            # if id==2:
            #     print()
            if min_dist<avoidance_radius and uav.get_mode()==VehicleMode.MISSION:  
                gcs.send_neighborg_position(uav,neighborg_id=jd)
                uav.set_mode(VehicleMode.AVOIDANCE)
            if uav.get_mode()==VehicleMode.AVOIDANCE and uav.current_step().state==State.DONE:
                uav.set_mode(VehicleMode.MISSION)
        uav.act_plan()

        if uav.plan.state == State.DONE:
            to_remove.add(id)
    gcs.update_missions(to_remove)
    

Vehicle 1: Plan 'basic' is now connected ✅🔗
Vehicle 1: ▶️ Starting Plan: basic
Vehicle 2: Plan 'hover' is now connected ✅🔗
Vehicle 2: ▶️ Starting Plan: hover
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: ▶️ Starting Action: Pre-Arm Check
Vehicle 2: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 2: ▶️ Starting Action: Pre-Arm Check
Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: Step 'Check disarmed' is now connected ✅🔗
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 2: Step 'Check EKF' is now connected ✅🔗
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: Step 'Check GPS' is now connected ✅🔗
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: Step 'Check GPS' is 

In [14]:
# def update_global_positions(mission_incomplete,global_poss):
#     for i in mission_incomplete:
#         uav=uavs[i]
#         current_action=uav.current_action().name 
#         if current_action in ['takeoff','fly']:
#             local_pos=get_local_position(uav.conn)
#             if local_pos is not False:
#                 global_poss[i] = local2global(local_pos,uav.home,pairwise=True)


#                 # if i == 1:
#                 #     global_poss[i] = spoofing_pos

# def avoid_object(uav,curr_pos):
#     uav.set_mode(VehicleMode.AVOIDANCE)
#     wp=uav.plan.current.current.wp
#     next_wp = get_avoidance_wp(pos=global2local(global_poss[i],uav.home,pairwise=True),
#                                 obj_pos=global2local(global_poss[j],uav.home,pairwise=True),
#                                 goal_pos=wp)
#     next_step=make_go_to(wp=next_wp,wp_margin=0.5,verbose=2,cause_text='(avoidance)')
#     uav.plan.current.add_now(next_step)

## Execute Plan

In [15]:

mission_incomplete = set(range(n_uavs))
global_poss = {}
while mission_incomplete:
    ## Get global positions
    update_global_positions(mission_incomplete,global_poss)
    global_poss[1] = spoofing_pos
    to_remove = set()
    for i in mission_incomplete:
        uav = uavs[i]
        if i in global_poss.keys():
            j,min_dist=get_nearest_neighbor(target_id=i,positions_dict=global_poss)
            if min_dist<avoidance_radius and uav.get_mode()==VehicleMode.MISSION:  
                uav.set_mode(VehicleMode.AVOIDANCE)
                wp=uav.plan.current.current.wp
                next_wp = get_avoidance_wp(pos=global2local(global_poss[i],uav.home,pairwise=True),
                                           obj_pos=global2local(global_poss[j],uav.home,pairwise=True),
                                           goal_pos=wp)
                next_step=make_go_to(wp=next_wp,wp_margin=0.5,verbose=2,cause_text='(avoidance)')
                uav.plan.current.add_now(next_step)
            if uav.get_mode()==VehicleMode.AVOIDANCE and uav.current_step().state==State.DONE:
                uav.set_mode(VehicleMode.MISSION)
        uav.act_plan()
        if uav.plan.state == State.DONE:
            to_remove.add(i)
            
    mission_incomplete -= to_remove  # batch-remove outside the loop

NameError: name 'update_global_positions' is not defined

In [ ]:
print(uavs[0].plan)

In [ ]:
print(uavs[1].plan)

## Kill all related process

In [ ]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")